In [23]:
import os
import re
from openai import OpenAI
from pydantic import BaseModel
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
import requests
from bs4 import BeautifulSoup
from Markdown2docx import Markdown2docx
from PIL import Image
API_KEY = os.getenv("API_KEY")

In [4]:
client = OpenAI(
    api_key=f"{API_KEY}",
    base_url="https://api.proxyapi.ru/openai/v1",
)

### Определение целевой аудитории и ее интересов

In [5]:
AUDIENCE = 'мамы'
DETAILS = 'Основной фокус на новости, касающиеся важности раннего обучения детей арифметике, скорочтению (это просто примеры, придумывай более разнообразные тематики) и другим предметам.'

### Генерация поисковых запросов на основе интересов аудитории

In [6]:
def generate_ideas(audience, details):
    prompt = f'Ты - ассистент, который должен находить в интернете последние новости, связанные с целевой аудиторией - {audience}. Сформируй 10 поисковых запросов для нахождения новостей, которыми интересуется данная аудитория. {DETAILS} Выводи только в виде списка, например: ["запрос1", "запрос2"].'
    chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
    )
    return chat_completion.choices[0].message.content

In [7]:
search_ideas = eval(generate_ideas(AUDIENCE, DETAILS))

In [8]:
search_ideas

['новости о раннем обучении детей',
 'значение раннего развития навыков чтения',
 'как помочь детям в освоении математики',
 'эффективные методики обучения детей арифметике',
 'польза скорочтения для школьников',
 'инновационные подходы к обучению детей дошкольного возраста',
 'новые исследования по детскому обучению и развитию',
 'как научить ребенка учиться самостоятельно',
 'тренды в образовательных методах для детей',
 'психология раннего образования: почему важен старт в школе']

### Поиск новостей по сгенерированным запросам

In [9]:
def search(query):
    search_engine = DuckDuckGoSearchResults()
    response = search_engine.run(query, backend="news")

    url_pattern = re.compile(r'https?://\S+')

    urls = re.findall(url_pattern, response)

    texts = re.split(url_pattern, response)

    results = [(text.strip()[:-7], url.rstrip(',')) for
               text, url in zip(texts, urls)]

    return results

In [10]:
results = [search(idea) for idea in search_ideas]
results_dict = {i : content for i, content in enumerate([item for sublist in results for item in sublist])}

### Извлечение текста из найденных новостей (пример)

In [11]:
def scrape_text_from_urls(urls):
    texts = []
    for url in urls:
        try:
            print(f"Scraping text from {url}...")
            response = requests.get(url)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            paragraphs = soup.find_all('p')

            text = 'n'.join([para.get_text() for para in paragraphs])
            
            if text:
                texts.append(text)

        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {e}")
            continue
        
    return texts

urls = [result[1] for result in results_dict.values()]

texts = scrape_text_from_urls(urls[:5])

Scraping text from https://kubnews.ru/obshchestvo/2024/10/22/sverdlovskiy-ombudsmen-predlozhil-priravnyat-detey-migrantov-k-invalidam/...
Scraping text from https://rtvi.com/news/sverdlovskij-ombudsmen-obyasnil-ideyu-obuchat-detej-migrantov-kak-invalidov/...
Scraping text from https://www.gazeta.ru/family/news/2024/10/11/24124033.shtml...
Scraping text from https://tagilcity.ru/news/2024-10-22/sverdlovskiy-ombudsmen-poyasnil-svoi-slova-ob-obuchenii-migrantov-na-pravah-invalidov-5228486...
Scraping text from https://progbasics.ru/blog/kak-naucit-rebyonka-pravilno-citat...
Error fetching https://progbasics.ru/blog/kak-naucit-rebyonka-pravilno-citat: 403 Client Error: Forbidden for url: https://progbasics.ru/blog/kak-naucit-rebyonka-pravilno-citat


### Формирование списка превью новостей

In [12]:
raw_text_list = []

for i, (key, value) in enumerate(results_dict.items()):
    snippet, _ = value
    raw_text = f"Сайт {i}: {snippet.strip('snippet: ')}"
    raw_text_list.append(raw_text)

raw_text_full = '/n'.join(raw_text_list)

### Выбор запросов для поста в блоге

In [16]:
chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": f"Проведи исследование, прочитав новости по сформированным запросам. Выяви запрос / несколько запросов, которые отлично подойдут для нового поста в твоем блоге (выведи их порядковый номер). Сделай акцент на новизну и актуальность информации для целевой аудитории российских {AUDIENCE}. Глубоко вздохни и, хорошенько подумав, напиши заголовок твоего поста. Выводи в json. Пример вывода: {{website_ids: [0, 2, 5], title: ['Заголовок твоего поста']}} Новости: " + raw_text_full}],
    response_format={ "type": "json_object" }
)

In [17]:
selector_output = eval(chat_completion.choices[0].message.content.replace('n  ', '').replace('n', ''))
selector_output

{'website_ids': [0, 2, 6, 34, 5],
 'title': ['Новые подходы к воспитанию: как современные тренды могут помочь в образовании ваших детей']}

### Написание поста в блоге

In [18]:
def write_post(selector_output):
    title = selector_output['title']
    selected_links = [results_dict.get(x)[1] for x in selector_output['website_ids']]
    scraped_data = 'Следующий сайт \n'.join(scrape_text_from_urls(selected_links))
    prompt = f'Представь, что ведешь свой блог. Используя собранные материалы и заголовок поста, напиши подходящий текст краткого поста. Вспомни, что ты пишешь как блогер с целевой аудиторией - {AUDIENCE}. Текст должен подходить для блога, иметь новизну и актуальность. Необязательно использовать всю информацию из материалов: сконцентрируйся на нескольких вещах и разбери их более детально, высказывая свое мнение, как это делает блогер. Не делай пост слишком затянутым. Используй emoji, где уместно. Веди себя наиболее естественно. Не пиши ничего, кроме поста, форматируй пост в стиле Markdown.'
    chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": f'Заголовок поста: {title}' + prompt + f'Вот собранные материалы: {scraped_data}'}])
    return chat_completion.choices[0].message.content

In [19]:
post = write_post(selector_output)

Scraping text from https://kubnews.ru/obshchestvo/2024/10/22/sverdlovskiy-ombudsmen-predlozhil-priravnyat-detey-migrantov-k-invalidam/...
Scraping text from https://www.gazeta.ru/family/news/2024/10/11/24124033.shtml...
Scraping text from https://guogagauzii.md/press-tsentr/novosti/item/3507-razvitie-yazykovykh-navykov-cherez-chtenie-strategii-kotorye-pozvolyat-povysit-effektivnost-mnogoyazychnogo-obrazovaniya-v-doshkolnykh-obrazovatelnykh-uchrezhdeniyakh-gagauzii...
Scraping text from https://myalma.ru/blog/stem---obrazovanie-v-dou-poslednie-resheniya-i-tendentsii/...
Scraping text from https://urok.1sept.ru/articles/701478...


### Сохранение поста в файл

In [22]:
with open('tmp.md', 'w+') as file:
    file.write(post)
project = Markdown2docx('tmp')
project.eat_soup()
project.save()

### Генерация обложки для поста с помощью DALL-E-3

In [ ]:
def create_post_association(post):
    prompt = 'На основе данного поста предложи главную ассоциацию, которую можно использовать для генерации картинки. n'
    chat_completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt + post}])
    return chat_completion.choices[0].message.content

In [ ]:
association = create_post_association(post)
print(association)

In [ ]:
def generate_cover(association_prompt):
  post_cover = client.images.generate(
    model="dall-e-3",
    prompt=association_prompt,
    size="1024x1024",
    quality="standard",
    n=1,
  )
  return post_cover

In [ ]:
post_cover = generate_cover(association)

In [ ]:
image = requests.get(post_cover.data[0].url).content
with open('image.png', 'wb') as f:
    f.write(image)

In [ ]:
Image.open('image.png')